In [ ]:
import pandas as pd
import re
import unicodedata
from datetime import datetime, timedelta
import json
xls = pd.ExcelFile("PhD Data Departments Wise (Updated).xlsx")
sheet_names = xls.sheet_names
all_sheets = {sheet: xls.parse(sheet) for sheet in sheet_names}

manual_replacements = {
    'admission category': 'Admission Category',
    'batch': 'Batch',
    'co-supervisor': 'Co-supervisor',
    'date of joining': 'Date of Joining',
    'department': 'Department',
    'designation of co-supervisor': 'Designation of Co-Supervisor',
    'designation of supervisor': 'Designation of Supervisor',
    'enrolment no': 'Enrolment No',
    'full time': 'Full Time/Part Time',
    'full time/part time': 'Full Time/Part Time',
    'full time/part-time': 'Full Time/Part Time',
    'gender': 'Gender',
    'registration no': 'Registration No',
    'remark': 'Remark',
    's.no': 'S.No',
    "student's name": "Student's Name",
    'supervisor': 'Supervisor'
}

column_names = [
    'S.No',
    "Student's Name",
    'Registration No',
    'Enrolment No',
    'Batch',
    'Date of Joining',
    'Department',
    'Gender',
    'Admission Category',
    'Full Time/Part Time',
    'Supervisor',
    'Designation of Supervisor',
    'Co-supervisor',
    'Designation of Co-Supervisor',
    'Remark'
]

all_columns = set()
new_df = pd.DataFrame(columns=column_names) #total 15 columns

for sheet, df in all_sheets.items():
    # Clean column names
    df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
    df.columns = df.columns.str.replace(r'\.1$', '', regex=True)  # Remove trailing '.1'
    df.columns = df.columns.str.replace(r'\.$', '', regex=True)  # Remove trailing dot (.)
    df.columns = df.columns.str.lower()  # Convert to lowercase
    df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
    df.columns = [manual_replacements.get(col, col) for col in df.columns]  # Apply manual replacements
    df = df.loc[:, ~df.columns.duplicated()]
    all_columns.update(df.columns)
    missing_columns = set(new_df.columns) - set(df.columns)  # Columns in new_df not in df
    for col in missing_columns:
        df[col] = pd.NA

    df = df[new_df.columns]
    new_df = pd.concat([new_df, df], ignore_index=True)
new_df['Date of Joining'] = pd.to_datetime(new_df['Date of Joining']).dt.date


combined_df=new_df

combined_df.drop(columns=['S.No', 'Designation of Supervisor', 'Designation of Co-Supervisor', 'Remark'], inplace=True, errors='ignore')

combined_df["Student's Name"] = combined_df["Student's Name"].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

for col in ['Registration No', 'Enrolment No']:
    combined_df[col] = combined_df[col].astype(str).str.replace(r'[^A-Za-z0-9]', '', regex=True).str.upper()

def excel_serial_to_date(val):
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(val))).date()
    except:
        return pd.NaT

def excel_serial_to_date(val):
    try:
        serial = float(val)
        if serial > 59:
            serial -= 1  # Excel incorrectly treats 1900 as a leap year, adjust for that
        return (datetime(1899, 12, 31) + timedelta(days=serial)).date()
    except:
        return pd.NaT
combined_df['Gender'] = combined_df['Gender'].astype(str).str.strip().str.lower().map(
    lambda g: 'M' if 'male' in g else 'F' if 'female' in g else 'Error'
)

combined_df['Full Time/Part Time'] = combined_df['Full Time/Part Time'].map(
    lambda t: '' if pd.isna(t) else 'FT' if 'full' in str(t).strip().lower() else 'PT' if 'part' in str(t).strip().lower() else '' if 'Female' else f'Error {t}'
)



def clean_supervisor(name):
    name = str(name).strip()
    name = re.sub(r'^Dr\.?\s*', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\s+', ' ', name)
    return '' if name.lower() == 'nil' else name.strip()

combined_df['Supervisor'] = combined_df['Supervisor'].apply(clean_supervisor)
combined_df['Co-supervisor'] = combined_df['Co-supervisor'].apply(clean_supervisor)

with open('/Users/rajesmanna/Documents/Phd/backend/conf.json', 'r') as file:
    data = json.load(file)
departments = data.get("college", {}).get("departments", {})
units = data.get("college", {}).get("administrative_units", {})
departments = {**departments, **units}
admission_category = data.get("admission_category", {})
def map_department(unit):
    if pd.isna(unit):
        return "ERROR: Missing unit"
    for key, value in departments.items():
        if str(unit).strip().lower() == str(value).strip().lower():
            return key
    return f"ERROR: {unit} is not a valid department"

# Map admission category function
def map_admission_category(category):
    if pd.isna(category):
        return "ERROR: Missing admission category"
    for key, value in admission_category.items():
        if str(category).strip().lower() == str(value).strip().lower():
            return key
    return f"ERROR: {category} is not a valid admission category"

fix_admission_category = {
    "Ph. D. Internal": "Ph.D. Internal",
    "Self- Financed": "Self Financed",
    "UGC JRF Sponsored ": "Sponsored UGC-JRF",
    "Sponsored (DST Inspire Fellowship)": "DST INSPIRE",
    "Open (Sponsord)": "Open (Sponsored)",
    "Sponsored-UGC-CSIR-JRF": "Sponsored (UGC-CSIR-JRF)"
}
combined_df["Admission Category"] = combined_df["Admission Category"].replace(fix_admission_category)

fix_department = {
    "Physics Department": "Physics",
    "Chemistry Department ": "Chemistry",
    "Electronics & Communication Engineering ": "Electronics and Communication Engineering",
    "Mathematics Department ": "Mathematics",
    "Metallurgical & Materials Engineering": "Materials and Metallurgical Engineering",
    "Humanities, Social Science & Management": "Humanities Social Sciences and Management",
    "Computer Science & Engineering":"Computer Science and Engineering"
}
combined_df["Department"] = combined_df["Department"].replace(fix_department)

combined_df["Department"] = combined_df["Department"].apply(map_department)
combined_df["Admission Category"] = combined_df["Admission Category"].apply(map_admission_category)
combined_df.to_excel("sholar.xlsx", index=False)
combined_df.head()






/var/folders/px/1stm6s094nlf81rx107bk0z40000gn/T/ipykernel_27229/1420051903.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.NA


,Student's Name,Registration No,Enrolment No,Batch,Date of Joining,Department,Gender,Admission Category,Full Time/Part Time,Supervisor,Co-supervisor
0,Ameer Ullah Ganai,2015FOEPHDAUTUMN48,2015FOEPHDAUTUMN48,2015,2015-08-31,CIVIL,M,SPON,PT,Mohammad Yousuf Shah,Bashir Ahmad Mir
1,Aliya Naseer,2017FOEPHDSPRING01,2017FOEPHDSPRING01,2017,2017-03-30,CIVIL,M,SPON,FT,Manzoor Ahmad Tantray,
2,Shakeel Ahmad Rather,2017FOEPHDSPRING08,2017FOEPHDSPRING08,2017,2017-03-30,CIVIL,M,SPON,FT,Javaid Ahmad Bhat,
3,Iftikhar Hussain Beigh,2017FOEPHDSPRING32,2017FOEPHDSPRING32,2017,2017-03-31,CIVIL,M,INST_FEL,FT,S Kaiser Bukhari,
4,Tariq Ahmad Sheikh,2017FOEPHDSPRING63,2017FOEPHDSPRING63,2017,2017-04-05,CIVIL,M,OPEN,PT,Jhon Mohammad Banday,Majid Hussain


In [12]:
import re
import unicodedata
import pandas as pd

# Update manual_replacements to handle variations like '.1' and spaces
manual_replacements = {
    'admission category': 'Admission Category',
    'batch': 'Batch',
    'co-supervisor': 'Co-supervisor',
    'date of joining': 'Date of Joining',
    'department': 'Department',
    'designation of co-supervisor': 'Designation of Co-Supervisor',
    'designation of supervisor': 'Designation of Supervisor',
    'enrolment no': 'Enrolment No',
    'full time': 'Full Time/Part Time',
    'full time/part time': 'Full Time/Part Time',
    'full time/part-time': 'Full Time/Part Time',
    'gender': 'Gender',
    'registration no': 'Registration No',
    'remark': 'Remark',
    's.no': 'S.No',
    "student's name": "Student's Name",
    'supervisor': 'Supervisor'
}

column_names = [
    'S.No',
    "Student's Name",
    'Registration No',
    'Enrolment No',
    'Batch',
    'Date of Joining',
    'Department',
    'Gender',
    'Admission Category',
    'Full Time/Part Time',
    'Supervisor',
    'Designation of Supervisor',
    'Co-supervisor',
    'Designation of Co-Supervisor',
    'Remark'
]

all_columns = set()
new_df = pd.DataFrame(columns=column_names) #total 15 columns

for sheet, df in all_sheets.items():
    # Clean column names
    df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
    df.columns = df.columns.str.replace(r'\.1$', '', regex=True)  # Remove trailing '.1'
    df.columns = df.columns.str.replace(r'\.$', '', regex=True)  # Remove trailing dot (.)
    df.columns = df.columns.str.lower()  # Convert to lowercase
    df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
    df.columns = [manual_replacements.get(col, col) for col in df.columns]  # Apply manual replacements
    df = df.loc[:, ~df.columns.duplicated()]
    all_columns.update(df.columns)
    missing_columns = set(new_df.columns) - set(df.columns)  # Columns in new_df not in df
    for col in missing_columns:
        df[col] = pd.NA

    df = df[new_df.columns]
    new_df = pd.concat([new_df, df], ignore_index=True)
new_df['Date of Joining'] = pd.to_datetime(new_df['Date of Joining']).dt.date






# print(len(all_columns))
new_df


/var/folders/px/1stm6s094nlf81rx107bk0z40000gn/T/ipykernel_27229/1706425984.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.NA


,S.No,Student's Name,Registration No,Enrolment No,Batch,Date of Joining,Department,Gender,Admission Category,Full Time/Part Time,Supervisor,Designation of Supervisor,Co-supervisor,Designation of Co-Supervisor,Remark
0,1,Ameer Ullah Ganai,2015-FOE-PH.D-AUTUMN-48,2015-FOE-PH.D-AUTUMN-48,2015,2015-08-31,Civil Engineering,Male,Sponsored,Part Time,Dr Mohammad Yousuf Shah,Professor,Dr Bashir Ahmad Mir,Professor,Admission on Hold
1,2,Aliya Naseer,2017-FOE-PH.D-SPRING-01,2017-FOE-PH.D-SPRING-01,2017,2017-03-30,Civil Engineering,Female,Sponsored,Full Time,Dr Manzoor Ahmad Tantray,Professor,Nil,Nil,NaN
2,3,Shakeel Ahmad Rather,2017-FOE-PH.D-SPRING-08,2017-FOE-PH.D-SPRING-08,2017,2017-03-30,Civil Engineering,Male,Sponsored,Full Time,Dr Javaid Ahmad Bhat,Professor,Nil,Nil,NaN
3,4,Iftikhar Hussain Beigh,2017-FOE-PH.D-SPRING-32,2017-FOE-PH.D-SPRING-32,2017,2017-03-31,Civil Engineering,Male,Institute Fellowship,Full Time,Dr S Kaiser Bukhari,Professor,Nil,Nil,NaN
4,5,Tariq Ahmad Sheikh,2017-FOE-PH.D-SPRING-63,2017-FOE-PH.D-SPRING-63,2017,2017-04-05,Civil Engineering,Male,Open,Part Time,Dr Jhon Mohammad Banday,Professor,Dr Majid Hussain,Assistant Professor,Admission on Hold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,50,Sumaira Gul,2022NITSGR0004,2022PHSCHM001,2022,2022-03-02,Chemistry Department,Female,Sponsored,Full Time,Prof Tabassum Ara,Professor,Nil,Nil,NaN
571,51,Syed Fozia,2024NITSGR1163,2024PHACHM013,2024,2024-09-24,Chemistry Department,Female,Self Financed,Full Time,Prof Kowsar Majid,Professor,Dr Faisal Qayoom Mir,Associate Professor,NaN
572,52,Tahmeena Akhter,2022NITSGR0264,2022PHACHM009,2022,2022-09-14,Chemistry Department,Female,Institute Fellowship,Full Time,Dr Mohammad Aslam,Assistant Professor,Nil,Nil,NaN
573,53,UMAR MOHI UD DIN BHAT,2023NITSGR546,2023PHACHM004,2023,2023-09-13,Chemistry Department,Male,Institute Fellowship,Full Time,Dr Javid Ahmad Banday,Professor,Dr S A Shah,Professor,NaN


# New Ph.D Scholars Data

In [34]:
# --- Initialization Cell ---

import pandas as pd
import numpy as np
import logging

# Setup logging
log_file = "scholars.log"
logging.basicConfig(filename=log_file, level=logging.INFO, filemode='w', force=True)

# Load Excel
df = pd.read_excel("/Users/rajesmanna/Documents/Phd/backend/media/Users/Ph. D Merged Data with Contact Details.xlsx")

# Drop unused columns if they exist

# --- Clean column names ---
# Remove leading/trailing/double spaces
df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
df.drop(columns=['S.No', 'Designation of Supervisor', 'Designation of Co-Supervisor', 'Remark'], inplace=True, errors='ignore')

df


,Salutation,Name,Gender,Date of Birth,Mobile Number,Personal Email,Type of Employee,Nature of Employment,Organization Unit,Post,Date of Joining,Official Email
0,Mr.,Muzaffar,Male,1965-05-01,9541047030,muzaffar123@gmail.com,Non-Teaching,Permanent,Department of Information Technology,Sr. Technician,1994-04-01,muzaffar123@gmail.com
1,Mr.,Mohammad Ashraf,Male,1972-05-05,7006984839,ashraf1232@gmail.com,Non-Teaching,Permanent,Department of Information Technology,Sr. Technician,1991-01-05,ashraf1232@gmail.com
2,Mr.,Manzoor,Male,1968-04-09,9797721521,manzoor.nit3@gmail.com,Non-Teaching,Permanent,Department of Information Technology,Sr. Technician,1991-01-04,manzoor.nit3@gmail.com
3,Ms.,Insha,Female,1993-05-25,7006752609,cseinsha@gmail.com,Non-Teaching,Permanent,ERP,Technical Assistant,2022-12-17,cseinsha@gmail.com
4,Mr.,Kalim Dar,Male,1988-05-16,9906481650,kaleem@nitsri.ac.in,Non-Teaching,Permanent,Department of Information Technology,Technical Assistant,2021-12-13,kaleem@nitsri.ac.in
...,...,...,...,...,...,...,...,...,...,...,...,...
468,Dr.,NaN,Female,NaN,3622315735,NaN,Teaching,Temporary,Department of Computer Science and Engineering,Assistant professor,NaN,ifrahraof21@gmail.com
469,Dr.,NaN,Female,NaN,7889473021,NaN,Teaching,Temporary,Department of Computer Science and Engineering,Assistant professor,NaN,mehkhan27@gmail.com
470,Dr.,NaN,Female,NaN,9541110299,NaN,Teaching,Temporary,Department of Computer Science and Engineering,Assistant professor,NaN,sadiahussain.hussain@gmail.com
471,Ms.,NaN,Female,NaN,9103595857,NaN,Non-Teaching,Temporary,Accounts,OFFICE ASSISTANT,NaN,hinarafiq@nitsri.ac.in


In [30]:
# --- Logging Cell ---

log = []

# --- Null Entries ---
log.append("\n--- Null Entries (Column-wise) ---")
null_counts = df.isnull().sum()
for col, count in null_counts.items():
    log.append(f"{col}: {count}")

# --- Duplicate Entries ---
log.append("\n--- Duplicate Entries (Column-wise) ---")
for col in df.columns:
    dup_series = df[col][df[col].duplicated(keep=False)]
    if not dup_series.empty:
        log.append(f"\nColumn: {col} - {dup_series.duplicated().sum()} duplicates")
        log.append(f"Duplicate values:\n{dup_series.dropna().unique().tolist()}")

# --- Unique Entries ---
log.append("\n--- Unique Entries (Column-wise) ---")
for col in df.columns:
    uniques = df[col].dropna().unique()
    log.append(f"\nColumn: {col} - {len(uniques)} unique values")
    log.append(f"Values: {uniques[:10]}{' ...' if len(uniques) > 10 else ''}")  # Limit to first 10

# Save logs to file
for entry in log:
    logging.info(entry)

print("Analysis complete. Results saved in 'scholars.log'.")


Analysis complete. Results saved in 'scholars.log'.


In [32]:
# --- Data Cleaning Cell ---

# 1. Normalize "Full Time/Part Time" column
df["Full Time/Part Time"] = df["Full Time/Part Time"].str.strip().replace({
    'Part-Time': 'Part Time',
    'Part-Time ': 'Part Time',
    'Part Time ': 'Part Time',
    'Full Time ': 'Full Time'
})

# 2. Normalize "Admission Category"
admission_mapping = {
    "Sponsored": [
        "Sponsored", "Sponsored ", "Sponsored-UGC-CSIR-JRF", "Sponsored UGC-JRF",
        "Sponsored (UGC-CSIR-JRF)"
    ],
    "Institute Fellowship": ["Institute Fellowship"],
    "Open": ["Open", "Open "],
    "Self Financed": ["Self Financed", "Self- Financed"],
    "Inspire DST Project": ["Inspire DST Project", "Inspire Dst Project"],
    "TEQIP Faculty": ["TEQIP Faculty "],
    "QIP": ["QIP"],
    "Sponsored Project": ["Sponsored Project"],
    "Ph.D. Internal": ["Ph. D. Internal", "Ph.D. Internal"],
    "DST INSPIRE": ["DST INSPIRE"]
}
# 5. Normalize 'Department' column

# Canonical department mapping (values only)
department_map = {
    "Chemical Engineering": "Chemical Engineering",
    "Chemistry Department": "Chemistry",
    "Civil Engineering": "Civil Engineering",
    "Computer Science & Engineering": "Computer Science and Engineering",
    "Electrical Engineering": "Electrical Engineering",
    "Electronics & Communication Engineering": "Electronics and Communication Engineering",
    "Humanities, Social Science & Management": "Humanities Social Sciences and Management",
    "Information Technology": "Information Technology",
    "Mathematics Department": "Mathematics",
    "Mechanical Engineering": "Mechanical Engineering",
    "Metallurgical & Materials Engineering": "Materials and Metallurgical Engineering",
    "Physics Department": "Physics"
}

# Clean up whitespace
df['Department'] = df['Department'].str.strip()

# Apply normalization
df['Department'] = df['Department'].replace(department_map)

df['Name of the Research Scholar'] = df['Name of the Research Scholar'].astype(str).str.replace(r'\s+', ' ', regex=True)


# Invert mapping: map original value to canonical
reverse_admission_map = {}
for k, values in admission_mapping.items():
    for v in values:
        reverse_admission_map[v.strip()] = k

df["Admission Category"] = df["Admission Category"].str.strip().replace(reverse_admission_map)

# 3. Clean 'Supervisor' names
df['Supervisor'] = df['Supervisor'].astype(str).str.replace(r'\s+', ' ', regex=True)
df['Supervisor'] = df['Supervisor'].str.replace(r'^(Dr\.?\s*|Prof\.?\s*)', '', regex=True)

# 4. Clean 'Co-supervisor' names
df['Co-supervisor'] = df['Co-supervisor'].astype(str).str.replace(r'\s+', ' ', regex=True)
df['Co-supervisor'] = df['Co-supervisor'].str.replace(r'^(Dr\.?\s*|Prof\.?\s*)', '', regex=True)
df['Co-supervisor'] = df['Co-supervisor'].replace(['Nil', 'Nil ',np.nan], '')

df


,Name of the Research Scholar,Department,Enrolment No.,Mail ID,Contact Number,Registration No,Batch,Gender,Full Time/Part Time,Date of Joining,Admission Category,Supervisor,Co-supervisor
0,Uzma Imtiyaz,Chemical Engineering,2019PHACHE006,NaN,NaN,2019NITSGR0901,2019,Female,Full Time,2019-12-06,Institute Fellowship,Mushtaq Ahmad Rather,nan
1,Nimaan Imtiyaz,Chemical Engineering,2019PHACHE007,nimaanmattoo94@gmail.com,7.780922e+09,2019NITSGR0917,2019,Male,Full Time,2019-12-06,Institute Fellowship,Tanveer Rasool,nan
2,Fayaz Doobi,Chemical Engineering,2019PHACHE009,ahmadfayaz264@gmail.com,9.682342e+09,2019NITSGR0963,2019,Male,Full Time,2020-01-22,Institute Fellowship,Faisal Qayoom Mir,nan
3,Gowhar Afzal,Chemical Engineering,2019PHACHE011,gowharafzal841@gmail.com,8.825055e+09,2019NITSGR0966,2019,Male,Full Time,2020-01-22,Institute Fellowship,Tanveer Rasool,nan
4,Massarat Majeed,Chemical Engineering,2019PHACHE012,massarat663@gmail.com,6.006166e+09,2019NITSGR0967,2019,Male,Full Time,2020-01-22,Institute Fellowship,Mushtaq Ahmad Rather,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,Ubaida Jan,Physics,2024PHSPHY004,ubaidulallah77@gmail.com,7.006589e+09,2024NITSGR011,2024,Female,Full Time,2024-05-20,Inspire DST Project,Mohd Ikram,nan
504,Masarat Fayaz Bhat,Physics,2024PHSPHY005,masrtfyzbhat@gmail.com,7.006817e+09,2024NITSGR012,2024,Male,Full Time,2024-06-12,DST INSPIRE,Seemin Rubab,nan
505,Faizan Hassan Shah,Physics,2024PHSPHY006,shahfaizan070@gmail.com,9.103134e+09,2024NITSGR014,2024,Male,Full Time,2024-07-05,DST INSPIRE,M A Shah,nan
506,Nazir Ahmad Ganaie,Physics,2025PHSPHY001,NaN,NaN,2025NITSGR0002,2025,Male,Part Time,2025-12-12,Sponsored,Seemin Rubab,nan


In [33]:
df.to_excel("sholar.xlsx", index=False)